In [2]:
import json
import pandas as pd
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor

## Load json (serial)

In [ ]:
raw_data = pd.DataFrame()

# Loop through years from 2018 to 2023
for year in range(2018, 2024):
    directory = f"../Data 2018-2023/Project/{year}/"

    # Ensure the directory exists before proceeding
    if not os.path.exists(directory):
        print(f"Directory {directory} does not exist.")
        continue

    # Get all JSON files in the directory
    json_files = [f for f in os.listdir(directory) if f.endswith("")]

    # Sort the files to ensure they are processed in order
    json_files.sort()

    for file_name in json_files:
        file_path = os.path.join(directory, file_name)
        print("Reading", file_path)

        if os.path.exists(file_path):
            try:
                with open(file_path, "r", encoding="utf-8") as file:
                    data = json.load(file)

                temp_df = pd.json_normalize(data)
                raw_data = pd.concat([raw_data, temp_df], ignore_index=True)
            except UnicodeDecodeError as e:
                print(f"UnicodeDecodeError while processing {file_path}: {e}")
            except json.JSONDecodeError as e:
                print(f"JSONDecodeError in file {file_path}: {e}")
        else:
            print(f"File {file_path} does not exist.")

## Load json (multithread)

In [ ]:
def load_to_dataframe(root_dir):
    """
    expected the path to contains only .json file
    """
    paths = []
    records = []

    def find_json_files(root_dir):
        """
        Recursively finds all JSON files in the given directory.
        """
        for dirpath, _, filenames in os.walk(root_dir):
            for file in filenames:
                paths.append(os.path.join(dirpath, file))

    def read_json_file(path):
        try:
            with open(path, "r", encoding="utf-8") as file:
                data = json.load(file)
            return pd.json_normalize(data).to_dict("records")[0]
        except UnicodeDecodeError as e:
            print(f"UnicodeDecodeError while processing {path}: {e}")
        except json.JSONDecodeError as e:
            print(f"JSONDecodeError in file {path}: {e}")
        return

    find_json_files(root_dir)

    with ThreadPoolExecutor() as executor:
        records = list(executor.map(read_json_file, paths))
        
    # Combine all DataFrames into a single DataFrame
    combined_df = pd.DataFrame(records)
    return combined_df

raw_data = load_to_dataframe("../Data 2018-2023/Project/")
raw_data.shape

In [ ]:
raw_data.to_csv("raw_data.csv")

## Data Exploration

In [ ]:
df = pd.DataFrame.copy(raw_data)
df.shape

In [ ]:
df.columns

In [ ]:
# Remove null columns
threshold = 0.5

max_null = 0

for col in df.columns:
    n_null = df[col].isnull().mean()

    if n_null > max_null:
        max_null = n_null
    
    if df[col].isnull().mean() > threshold:
        df.drop(columns=col, axis=1, inplace=True)
        
print(max_null)

In [ ]:
df.columns

In [ ]:
# Remove useless columns

useless_cols = [
    "abstracts-retrieval-response.item.ait:process-info.ait:status.@type",
    "abstracts-retrieval-response.item.ait:process-info.ait:status.@state",
    "abstracts-retrieval-response.item.ait:process-info.ait:status.@stage",  # just status
    "abstracts-retrieval-response.item.ait:process-info.ait:date-delivered.@day",
    "abstracts-retrieval-response.item.ait:process-info.ait:date-delivered.@timestamp",
    "abstracts-retrieval-response.item.ait:process-info.ait:date-delivered.@month",  # just year is enough
    "abstracts-retrieval-response.item.ait:process-info.ait:date-sort.@day",
    "abstracts-retrieval-response.item.ait:process-info.ait:date-sort.@month",
    "abstracts-retrieval-response.item.bibrecord.head.abstracts",
    "abstracts-retrieval-response.item.bibrecord.head.correspondence.affiliation.@country",  # is short form of country
    "abstracts-retrieval-response.item.bibrecord.head.citation-info.citation-type.@code",  # just type
    "abstracts-retrieval-response.item.bibrecord.head.citation-info.citation-language.@xml:lang",  # short form of language
    "abstracts-retrieval-response.item.bibrecord.head.source.sourcetitle-abbrev",  # title not needed
    "abstracts-retrieval-response.item.bibrecord.head.source.website.ce:e-address.$",  # website
    "abstracts-retrieval-response.item.bibrecord.head.source.website.ce:e-address.@type",  # type of above
    "abstracts-retrieval-response.item.bibrecord.head.source.volisspag.pagerange.@first",
    "abstracts-retrieval-response.item.bibrecord.head.source.volisspag.pagerange.@last",  # could be good if we do how much pages ref prediction
    "abstracts-retrieval-response.item.bibrecord.head.source.@type",  # just type
    "abstracts-retrieval-response.item.bibrecord.head.source.sourcetitle",  # just title
    "abstracts-retrieval-response.item.bibrecord.head.source.@srcid",  # id
    "abstracts-retrieval-response.item.bibrecord.head.source.publicationdate.month",
    "abstracts-retrieval-response.item.bibrecord.head.source.publicationdate.year",  # redundant publish year
    "abstracts-retrieval-response.item.bibrecord.head.source.publicationdate.date-text.@xfab-added",
    "abstracts-retrieval-response.item.bibrecord.head.source.publicationdate.date-text.$",
    "abstracts-retrieval-response.item.bibrecord.head.source.publicationdate.day",  # just year is enough
    "abstracts-retrieval-response.item.bibrecord.head.correspondence.person.ce:initials", # useless first char name
    "abstracts-retrieval-response.item.bibrecord.item-info.copyright.$",
    "abstracts-retrieval-response.item.bibrecord.item-info.copyright.@type",
    "abstracts-retrieval-response.item.bibrecord.item-info.history.date-created.@day",
    "abstracts-retrieval-response.item.bibrecord.item-info.history.date-created.@timestamp",
    "abstracts-retrieval-response.item.bibrecord.item-info.history.date-created.@year",
    "abstracts-retrieval-response.item.bibrecord.item-info.history.date-created.@month",  # why it created in 2020 when it is 2018 file 💀
    "abstracts-retrieval-response.item.bibrecord.item-info.itemidlist.itemid",
    "abstracts-retrieval-response.item.bibrecord.item-info.itemidlist.ce:doi",  # ids
    "abstracts-retrieval-response.coredata.srctype",  # just type
    "abstracts-retrieval-response.coredata.eid",  # ids
    "abstracts-retrieval-response.coredata.prism:url",
    "abstracts-retrieval-response.coredata.subtypeDescription",
    "abstracts-retrieval-response.coredata.link",
    "abstracts-retrieval-response.coredata.source-id",
    "abstracts-retrieval-response.coredata.prism:endingPage",
    "abstracts-retrieval-response.coredata.openaccess",
    "abstracts-retrieval-response.coredata.openaccessFlag",  # not necessary
    "abstracts-retrieval-response.coredata.prism:doi",
    "abstracts-retrieval-response.coredata.prism:startingPage",
    "abstracts-retrieval-response.coredata.subtype",
    "abstracts-retrieval-response.coredata.dc:identifier",  # also id
    "abstracts-retrieval-response.coredata.publishercopyright",
    # "abstracts-retrieval-response.coredata.dc:publisher",
    "abstracts-retrieval-response.language.@xml:lang",
    "abstracts-retrieval-response.authors.author",  # redundant author
    "abstracts-retrieval-response.item.xocs:meta.xocs:funding-list.@pui-match",  # what is this?
    "abstracts-retrieval-response.item.xocs:meta.xocs:funding-list.@has-funding-info",
    "abstracts-retrieval-response.item.xocs:meta.xocs:funding-list.xocs:funding-addon-generated-timestamp",
    "abstracts-retrieval-response.item.xocs:meta.xocs:funding-list.xocs:funding-addon-type",  # link
    "abstracts-retrieval-response.item.bibrecord.head.citation-info.abstract-language.@xml:lang",  # redundant with language
    "abstracts-retrieval-response.item.bibrecord.head.source.translated-sourcetitle.$", # title
    "abstracts-retrieval-response.item.bibrecord.head.source.volisspag.voliss.@volume",
    "abstracts-retrieval-response.item.bibrecord.head.source.issn",  # id?
    "abstracts-retrieval-response.coredata.dc:description",
    "abstracts-retrieval-response.coredata.prism:volume",
    "abstracts-retrieval-response.coredata.prism:issn",
    "abstracts-retrieval-response.item.xocs:meta.xocs:funding-list.xocs:funding-text",  # basically desc
    "abstracts-retrieval-response.item.bibrecord.head.correspondence.affiliation.postal-code",
    "abstracts-retrieval-response.item.bibrecord.head.citation-info.author-keywords.author-keyword",  # probably too many unique
    "abstracts-retrieval-response.authkeywords.author-keyword",  # probably too many unique
    "abstracts-retrieval-response.item.bibrecord.head.source.volisspag.voliss.@issue",
    "abstracts-retrieval-response.coredata.prism:issueIdentifier",
    "abstracts-retrieval-response.coredata.prism:pageRange",
    "abstracts-retrieval-response.item.xocs:meta.xocs:funding-list.xocs:funding", # we donot care funding
    "abstracts-retrieval-response.item.bibrecord.head.grantlist.@complete",
    "abstracts-retrieval-response.item.bibrecord.head.grantlist.grant-text.$",
    "abstracts-retrieval-response.item.bibrecord.head.grantlist.grant-text.@xml:lang",
    "abstracts-retrieval-response.idxterms.mainterm",
    "abstracts-retrieval-response.item.bibrecord.head.source.translated-sourcetitle.@xml:lang",
]

df.drop(columns=useless_cols, axis=1, inplace=True)

In [11]:
df.columns

Index(['abstracts-retrieval-response.item.ait:process-info.ait:date-delivered.@year',
       'abstracts-retrieval-response.item.ait:process-info.ait:date-sort.@year',
       'abstracts-retrieval-response.item.bibrecord.head.author-group',
       'abstracts-retrieval-response.item.bibrecord.head.citation-title',
       'abstracts-retrieval-response.item.bibrecord.head.correspondence.affiliation.country',
       'abstracts-retrieval-response.item.bibrecord.head.correspondence.affiliation.city',
       'abstracts-retrieval-response.item.bibrecord.head.correspondence.affiliation.organization',
       'abstracts-retrieval-response.item.bibrecord.head.correspondence.person.ce:given-name',
       'abstracts-retrieval-response.item.bibrecord.head.correspondence.person.ce:surname',
       'abstracts-retrieval-response.item.bibrecord.head.correspondence.person.ce:indexed-name',
       'abstracts-retrieval-response.item.bibrecord.head.citation-info.citation-language.@language',
       'abstracts-

In [99]:
column_renaming = {
    "abstracts-retrieval-response.item.ait:process-info.ait:date-delivered.@year": "date_delivered_year",
    "abstracts-retrieval-response.item.ait:process-info.ait:date-sort.@year": "date_sort_year",
    "abstracts-retrieval-response.item.bibrecord.head.author-group": "author_group",
    "abstracts-retrieval-response.item.bibrecord.head.citation-title": "citation_title",
    # "abstracts-retrieval-response.item.bibrecord.head.abstracts": "abstracts",
    "abstracts-retrieval-response.item.bibrecord.head.correspondence.affiliation.country": "affiliation_country",
    "abstracts-retrieval-response.item.bibrecord.head.correspondence.affiliation.city": "affiliation_city",
    "abstracts-retrieval-response.item.bibrecord.head.correspondence.affiliation.organization": "affiliation_organization",
    "abstracts-retrieval-response.item.bibrecord.head.correspondence.person.ce:given-name": "corresponding_author_given_name",
    # "abstracts-retrieval-response.item.bibrecord.head.correspondence.person.ce:initials": "corresponding_author_initials",
    "abstracts-retrieval-response.item.bibrecord.head.correspondence.person.ce:surname": "corresponding_author_surname",
    "abstracts-retrieval-response.item.bibrecord.head.correspondence.person.ce:indexed-name": "corresponding_author_indexed_name",
    "abstracts-retrieval-response.item.bibrecord.head.citation-info.citation-language.@language": "citation_language",
    "abstracts-retrieval-response.item.bibrecord.head.citation-info.abstract-language.@language": "abstract_language",
    "abstracts-retrieval-response.item.bibrecord.head.source.@country": "source_country",
    # "abstracts-retrieval-response.item.bibrecord.head.source.translated-sourcetitle.@xml:lang": "source_translated_title_lang",
    "abstracts-retrieval-response.item.bibrecord.head.source.publicationyear.@first": "source_publication_year",
    "abstracts-retrieval-response.item.bibrecord.head.source.publisher.publishername": "source_publisher_name",
    "abstracts-retrieval-response.item.bibrecord.head.enhancement.classificationgroup.classifications": "classificationgroup",
    "abstracts-retrieval-response.item.bibrecord.item-info.dbcollection": "dbcollection",
    "abstracts-retrieval-response.item.bibrecord.tail.bibliography.@refcount": "ref_count",
    "abstracts-retrieval-response.item.bibrecord.tail.bibliography.reference": "reference",
    "abstracts-retrieval-response.affiliation": "affiliation",
    "abstracts-retrieval-response.coredata.prism:coverDate": "coverDate",
    "abstracts-retrieval-response.coredata.prism:aggregationType": "aggregationType",
    "abstracts-retrieval-response.coredata.dc:creator.author": "author",
    "abstracts-retrieval-response.coredata.prism:publicationName": "publicationName",
    "abstracts-retrieval-response.coredata.citedby-count": "citedby_count",
    # "abstracts-retrieval-response.coredata.subtype": "subtype",
    # "abstracts-retrieval-response.coredata.prism:pageRange": "pageRange",
    "abstracts-retrieval-response.coredata.dc:title": "title",
    "abstracts-retrieval-response.subject-areas.subject-area": "subject_area",
    # "abstracts-retrieval-response.coredata.publishercopyright": "publishercopyright",
    "abstracts-retrieval-response.coredata.dc:publisher": "publisher",
    # "abstracts-retrieval-response.idxterms.mainterm": "mainterm",
}

df.rename(columns=column_renaming, inplace=True)

In [ ]:
# df.dropna(axis=0, inplace=True)

for col in df.columns:
    # if (df[col].isnull()):
    print(col, df[col].isnull().mean())

In [ ]:
df.info()

In [15]:
# important col
df['subject_area'] = df['subject_area'].fillna('Unknown')

In [16]:
# convert to datetime
df['coverDate'] = pd.to_datetime(df['coverDate'], errors='coerce')

In [ ]:
df.info()

In [18]:
# Convert columns to numeric
df['source_publication_year'] = pd.to_numeric(df['source_publication_year'], errors='coerce')
df['date_sort_year'] = pd.to_numeric(df['date_sort_year'], errors='coerce')
df['date_delivered_year'] = pd.to_numeric(df['date_delivered_year'], errors='coerce')

df['citedby_count'] = pd.to_numeric(df['citedby_count'], errors='coerce')
df['ref_count'] = pd.to_numeric(df['ref_count'], errors='coerce')

# Fill NaN with the mean value and convert to integer type
df['citedby_count'] = df['citedby_count'].fillna(df['citedby_count'].mean()).astype(int)
df['ref_count'] = df['ref_count'].fillna(df['ref_count'].mean()).astype(int)

In [ ]:
df.info()

In [20]:
df.fillna("Unknown", inplace=True)